In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup
from glob import glob
import json, requests


In [2]:
def downloadCDCdata():
    # front end CDC page with links
    URL = 'https://beta.healthdata.gov/National/COVID-19-Community-Profile-Report/gqxm-d9w9'

    # download Page as HTML
    page = requests.get(URL)

    # parse HTML with BS4
    soup = BeautifulSoup(page.content, 'html.parser')

    # find the script block with the URLs we need
    for a in soup.find_all('script'):
        if 'initialState' in str(a):
            linkBlock = str(a)[69:-17].split('},{')

    # declare list to be populated
    cleanedLinks = []

    # pull out clean links to list
    # the reason for this is that the links are not just dates --
    # each has a UUID we need to scrape in order to download the excel
    for link in linkBlock:
        cleaned = link.split('"href":"')[-1].split(',"link"')[0]
        if ('Community_Profile_Report' in cleaned) and ('/api/views/' in cleaned) and ('Public.xlsx' in cleaned) and (len(cleaned) < 300):
            cleanedLinks.append(f"https://beta.healthdata.gov{cleaned}")

    # check out current excel files
    downloadFiles = glob('./xlsx/*.xlsx')

    # download any files we're missing in the xlsx folder
    for url in cleanedLinks:
        r = requests.get(url)
        date = url[-21:-13]
        if f"./xlsx\\CDC_{url[-21:-13]}.xlsx" not in downloadFiles:
            with open(f"./xlsx/CDC_{date}.xlsx", 'wb') as f:
                f.write(r.content)

In [3]:
def findColumn(columns, search):
    for column in columns:
        if search.lower() in column.lower():
            return column
    return None

In [4]:
def parseCDCdata(fileList):
    for index, file in enumerate(fileList):
        # import excel sheets
        raw = pd.read_excel(file, sheet_name="Counties")
        # snag case and testing dates
        endOfWeekCases = f"{raw.columns[14].split('(')[-1].split(' ')[0]} {raw.columns[14].split('-')[-1][:-1]}"
        endofWeekCases = datetime.strptime(endOfWeekCases, '%B %d')
        endOfWeekTesting = f"{raw.columns[32].split('(')[-1].split(' ')[0]} {raw.columns[32].split('-')[-1][:-1]}"
        endOfWeekTesting = datetime.strptime(endOfWeekTesting, '%B %d')
        year = file[-13:-9]
        # set second row as column index
        # this is to handle the CDC excel formatting
        raw.columns = list(raw.iloc[0])
        raw = raw.drop(index=0)
        columns = list(raw.columns)


        cases = raw[['FIPS code', findColumn(list(raw.columns), 'cumulative cases')]]
        cases.columns = ['FIPS', f"{endofWeekCases.month}/{endofWeekCases.day}/{year}"]   
        
        deaths = raw[['FIPS code',findColumn(list(raw.columns), 'cumulative deaths')]]
        cases.columns = ['FIPS', f"{endofWeekCases.month}/{endofWeekCases.day}/{year}"]

        testingPos = raw[['FIPS code',  findColumn(list(raw.columns), 'test positivity rate - last 7 day')]]
        testingPos.columns = ['FIPS', f"{endOfWeekTesting.month}/{endOfWeekTesting.day}/{year}"]

        testing = raw[['FIPS code', findColumn(list(raw.columns), 'Total RT-PCR diagnostic tests - last 7 days')]]
        testing.columns = ['FIPS', f"{endOfWeekTesting.month}/{endOfWeekTesting.day}/{year}"]

        testingCap = raw[['FIPS code', findColumn(list(raw.columns), 'RT-PCR tests per 100k - previous 7 days')]]
        testingCap.columns = ['FIPS', f"{endOfWeekTesting.month}/{endOfWeekTesting.day}/{year}"]

        testingCcpt = raw[['FIPS code',  findColumn(list(raw.columns), 'cases - last 7 days'),  findColumn(list(raw.columns), 'Total RT-PCR diagnostic tests - last 7 days')]]
        testingCcpt.columns = ['FIPS', 'Cases', 'Tests']

        testingCcptFiltered = testingCcpt.query('Tests > 0')
        testingCcptFiltered['CCPT'] = testingCcptFiltered['Cases']/testingCcptFiltered['Tests']

        testingCcpt = testingCcpt.merge(testingCcptFiltered, on=['FIPS','Cases','Tests'], how="left")[['FIPS','CCPT']]
        testingCcpt.columns = ['FIPS', f"{endOfWeekTesting.month}/{endOfWeekTesting.day}/{year}"]

        prevCases = cases.copy()
        prevDeaths = deaths.copy()
        prevTesting = testing.copy()
        prevTestingPos = testingPos.copy()
        prevTestingCap = testingCap.copy()
        prevTestingCcpt = testingCcpt.copy()

        if index != 0:
            if list(cases.columns)[1] not in list(prevCases.columns):
                prevCases = prevCases.merge(cases, on="FIPS", how="outer")
            if list(deaths.columns)[1] not in list(prevDeaths.columns):
                prevDeaths = prevDeaths.merge(deaths, on="FIPS", how="outer")
            if list(testing.columns)[1] not in list(prevTesting.columns):
                prevTesting = prevTesting.merge(testing, on="FIPS", how="outer")
            if list(testingPos.columns)[1] not in list(prevTestingPos.columns):
                prevTestingPos = prevTestingPos.merge(testingPos, on="FIPS", how="outer")
            if list(testingCap.columns)[1] not in list(prevTestingCap.columns):
                prevTestingCap = prevTestingCap.merge(testingCap, on="FIPS", how="outer")
            if list(testingCcpt.columns)[1] not in list(prevTestingCcpt.columns):
                prevTestingCcpt = prevTestingCcpt.merge(testingCcpt, on="FIPS", how="outer")
    
    prevCases.to_csv('./csv/covid_confirmed_cdc.csv',index=False)
    prevDeaths.to_csv('./csv/covid_deaths_cdc.csv',index=False)
    prevTesting.to_csv('./csv/covid_testing_cdc.csv',index=False)
    prevTestingPos.to_csv('./csv/covid_wk_pos_cdc.csv',index=False)
    prevTestingCap.to_csv('./csv/covid_tcap_cdc.csv',index=False)
    prevTestingCcpt.to_csv('./csv/covid_ccpt_cdc.csv',index=False)

In [180]:
downloadCDCdata()
parseCDCdata(glob('./xlsx/*.xlsx'))

./xlsx\CDC_20201217.xlsx
1900-12-16 00:00:00
1900-12-14 00:00:00
./xlsx\CDC_20201218.xlsx


<ipython-input-179-446044abee93>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testingCcptFiltered['CCPT'] = testingCcptFiltered['Cases']/testingCcptFiltered['Tests']


1900-12-17 00:00:00
1900-12-15 00:00:00
./xlsx\CDC_20201219.xlsx
1900-12-18 00:00:00
1900-12-16 00:00:00
./xlsx\CDC_20201220.xlsx
1900-12-19 00:00:00
1900-12-17 00:00:00
./xlsx\CDC_20201221.xlsx
1900-12-20 00:00:00
1900-12-18 00:00:00
./xlsx\CDC_20201222.xlsx
1900-12-21 00:00:00
1900-12-19 00:00:00
./xlsx\CDC_20201223.xlsx
1900-12-22 00:00:00
1900-12-20 00:00:00
./xlsx\CDC_20201226.xlsx
1900-12-25 00:00:00
1900-12-23 00:00:00
